# 0. 개요

- 모든 시간대를 대상으로 하는 것이 아닌, 예측해야하는 시간대의 target이 비슷한 데이터를 바탕으로
- 2016-04-25 ~ 2016-05-22 : d_1914 ~ d_1941 (public)
- 2016-05-23 ~ 2016-06-19 : d_1942 ~ d_1969 (private)
- 모델을 제작

# 1. 라이브러리 로드 및 데이터 불러오기

In [ ]:
import pandas as pd
import numpy as np 
from sklearn.ensemble import RandomForestRegressor
from lightgbm import LGBMRegressor
from sklearn import metrics
from sklearn.model_selection import TimeSeriesSplit, KFold

import matplotlib.pyplot as plt
import seaborn as sns
import pickle 
import time
import datetime
import os

from scipy.sparse import csr_matrix

from mypackage import *

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 400)

### <div style="color:red">파일명 체크!!!</div>
- all_df: 기초작업 수준. 
- all_df4: lag 데이터 엄청 많이 만들어 놓은 것.
- all_df5: 'id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id', 'd', 'sales', 'date', 'wm_yr_wk', 'weekday', 'month', 'year', 'event_name_1', 'event_type_1', 'snap_CA', 'snap_TX', 'snap_WI', 'is_event', 'day', 'week', 'sell_price', 'revenue', sales_rolling_mean_t7', 'rolling_mean_t7', 'rolling_std_t7', 'sales_rolling_mean_t28', 'rolling_mean_t28', 'rolling_std_t28', 'sales_rolling_mean_t56', 'rolling_mean_t56', 'rolling_std_t56', 'sales_rolling_mean_t112','rolling_mean_t112', 'rolling_std_t112', 'sales_rolling_mean_t168', 'rolling_mean_t168', 'rolling_std_t168', 'lag_t28', 'lag_t29', 'lag_t30', 'lag_t31', 'lag_t32', 'lag_t33', 'lag_t34', 'revenue_lag_t28', 'revenue_lag_t29', 'revenue_lag_t30','revenue_lag_t31', 'revenue_lag_t32', 'revenue_lag_t33', 'revenue_lag_t34'

In [ ]:
print('data loading')
with open('inputs/all_df_eval.pickle', 'rb') as f:
    all_df = pickle.load(f)
print('data loaded')

In [ ]:
all_df.head()

In [ ]:
for i in range(33, 43):
    all_df[f'lag_t{i}'] = all_df.groupby(['id'])['sales'].transform(lambda x: x.shift(i))

In [ ]:
all_df['weekofmonth'] = np.ceil(all_df['day'] // 7).astype('int8')

In [ ]:
all_df.columns

In [ ]:
all_df = all_df[['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id', 'd',
       'sales', 'date', 'wm_yr_wk', 'weekday', 'month', 'year', 'event_name_1',
       'event_type_1', 'snap_CA', 'snap_TX', 'snap_WI', 'is_event', 'day',
       'week', 'sell_price', 'lag_t28', 'lag_t29', 'lag_t30',
       'lag_t31', 'lag_t32', 'rolling_mean_t7', 'rolling_std_t7',
       'rolling_mean_t28', 'rolling_std_t28', 'rolling_mean_t56',
       'rolling_std_t56', 'rolling_mean_t84', 'rolling_std_t84',
       'rolling_mean_t112', 'rolling_std_t112', 'rolling_mean_t168',
       'rolling_std_t168', 'price_change_t1', 'price_change_t365',
       'rolling_price_std_t7', 'rolling_price_std_t28', 'lag_t33', 'lag_t34',
       'lag_t35', 'lag_t36', 'lag_t37', 'lag_t38', 'lag_t39', 'lag_t40',
       'lag_t41', 'lag_t42', 'weekofmonth']]

In [ ]:
# grid_df['rolling_mean_t7'] = grid_df.groupby(['id'])['sales'].transform(lambda x: x.shift(28).rolling(7).mean())
# grid_df['rolling_std_t7'] = grid_df.groupby(['id'])['sales'].transform(lambda x: x.shift(28).rolling(7).std())

# 새롭게 만들거
# grid_df['rolling_mean_t32'] = grid_df.groupby(['id'])['sales'].transform(lambda x: x.shift(28).rolling(32).mean())
# grid_df['rolling_std_t32'] = grid_df.groupby(['id'])['sales'].transform(lambda x: x.shift(28).rolling(32).std())

# grid_df['rolling_mean_t64'] = grid_df.groupby(['id'])['sales'].transform(lambda x: x.shift(28).rolling(64).mean())
# grid_df['rolling_std_t64'] = grid_df.groupby(['id'])['sales'].transform(lambda x: x.shift(28).rolling(64).std())

# grid_df['rolling_mean_t96'] = grid_df.groupby(['id'])['sales'].transform(lambda x: x.shift(28).rolling(96).mean())
# grid_df['rolling_std_t96'] = grid_df.groupby(['id'])['sales'].transform(lambda x: x.shift(28).rolling(96).std())

# grid_df['rolling_mean_t180'] = grid_df.groupby(['id'])['sales'].transform(lambda x: x.shift(28).rolling(180).mean())
# grid_df['rolling_std_t180'] = grid_df.groupby(['id'])['sales'].transform(lambda x: x.shift(28).rolling(180).std())


# grid_df['rolling_mean_t30'] = grid_df.groupby(['id'])['sales'].transform(lambda x: x.shift(28).rolling(30).mean())
# grid_df['rolling_std_t30'] = grid_df.groupby(['id'])['sales'].transform(lambda x: x.shift(28).rolling(30).std())


# grid_df['rolling_mean_t90'] = grid_df.groupby(['id'])['sales'].transform(lambda x: x.shift(28).rolling(90).mean())
# grid_df['rolling_std_t90'] = grid_df.groupby(['id'])['sales'].transform(lambda x: x.shift(28).rolling(90).std())

# grid_df['rolling_mean_t120'] = grid_df.groupby(['id'])['sales'].transform(lambda x: x.shift(28).rolling(120).mean())
# grid_df['rolling_std_t120'] = grid_df.groupby(['id'])['sales'].transform(lambda x: x.shift(28).rolling(120).std())

# grid_df['rolling_mean_t180'] = grid_df.groupby(['id'])['sales'].transform(lambda x: x.shift(28).rolling(180).mean())
# grid_df['rolling_std_t180'] = grid_df.groupby(['id'])['sales'].transform(lambda x: x.shift(28).rolling(180).std())


# 2. 데이터 탐색

여기서는 test셋과 비슷한 시간대를 찾아서, 이 데이터만을 바탕으로 모델을 만들것임.  

test: 2016-04-25 ~ 2016-05-22  17~20주차 (16주차나 21주차까지 늘려야될지도 생각해보자)

---
train: 2015-04-27 ~ 2015-05-24  
train: 2014-04-28 ~ 2014-05-25  / 2014-04-21 ~ 2014-05-18
train: 2013-04-27 ~ 2013-05-24  
train: 2012-04-27 ~ 2012-05-24 
train: 2011-04-27 ~ 2011-05-24 

In [ ]:
len(all_df)

In [ ]:
all_df.tail()

In [ ]:
# c00 = all_df['week'] == 9
# c01 = all_df['week'] == 10
# c02 = all_df['week'] == 11
# c03 = all_df['week'] == 12

c1 = all_df['week'] == 17
c2 = all_df['week'] == 18
c3 = all_df['week'] == 19
c4 = all_df['week'] == 20

c5 = all_df['week'] == 21
c6 = all_df['week'] == 22
c7 = all_df['week'] == 23
c8 = all_df['week'] == 24

all_df2 = all_df[c1 | c2 | c3 | c4 | c5 | c6 | c7 | c8]
# all_df2 = all_df[c1 | c2 | c3 | c4 | c5 | c6 | c7 | c8 |c00|c01|c02|c03]

In [ ]:
len(all_df2)

In [ ]:
item_id_gb_sell_price_mean = all_df2.groupby(['id', 'year', 'week'])['sell_price'].mean().rename('item_id_gb_sell_price_mean')
all_df2 = pd.merge(all_df2, item_id_gb_sell_price_mean, on=['id', 'year', 'week'], how='left')


item_id_gb_sell_price_std = all_df2.groupby(['id', 'year', 'week'])['sell_price'].std().rename('item_id_gb_sell_price_std')
all_df2 = pd.merge(all_df2, item_id_gb_sell_price_std, on=['id', 'year', 'week'], how='left')

In [ ]:
from sklearn.preprocessing import LabelEncoder

le_id = LabelEncoder()
le_id.fit(all_df2['id'])

all_df2['id'] = le_id.transform(all_df2['id'])

In [ ]:
all_df2.columns

# 3. 피쳐 엔지니어링

## 3.1 train, test

prepare training and test data.
- 2011-01-29 ~ 2016-04-24 : d_1    ~ d_1913
- 2016-04-25 ~ 2016-05-22 : d_1914 ~ d_1941 (public)
- 2016-05-23 ~ 2016-06-19 : d_1942 ~ d_1969 (private)

# 4. 모델 제작

## 4.1 train set /  test set 선정

feature 선정에 있어서 다른 노트북에서는 all_df = all_df.drop(['rolling_price_max_t365', 'lag_price_t1'], inplace = True, axis = 1) 이렇게 진행했음

In [ ]:
# all_df2 = all_df2.rename(columns={'target':'sales'})

2011-01-29 ~ 2016-04-24 : d_1 ~ d_1913  
2016-04-25 ~ 2016-05-22 : d_1914 ~ d_1941 (public)  
2016-05-23 ~ 2016-06-19 : d_1942 ~ d_1969 (private)  

In [ ]:
DROP_COLS = ['sales', 'date', 'wm_yr_wk']

In [ ]:
train_set_X = all_df2[all_df2['date'] <= '2016-05-22']
# train_weights = train_set['weight'].copy()
# train_set_X = train_set[features]
train_set_y = train_set_X['sales']
train_set_X = train_set_X.drop(DROP_COLS, axis=1)

# 테스트 셋
test_set = all_df2[all_df2['date'] > '2016-05-22']
test_set = test_set.drop(DROP_COLS, axis=1)


var_set_X = all_df2[(all_df2['date'] > '2015-05-22') & (all_df2['date'] <= '2015-06-19')]
var_set_y = var_set_X['sales']
var_set_X = var_set_X.drop(DROP_COLS, axis=1)

In [ ]:
train_set_X.head()

In [ ]:
n_fold = 8
folds = KFold(n_splits=n_fold, shuffle=True)
splits = folds.split(train_set_X, train_set_y)

y_preds = np.zeros(test_set.shape[0])
y_oof = np.zeros(train_set_X.shape[0])

feature_importances = pd.DataFrame()
feature_importances['feature'] = train_set_X.columns
mean_score = []

print(train_set_X.columns)

for fold_n, (train_index, valid_index) in enumerate(splits):
    print('Fold:',fold_n+1)
    
    X_train, X_valid = train_set_X.iloc[train_index], train_set_X.iloc[valid_index]
    y_train, y_valid = train_set_y.iloc[train_index], train_set_y.iloc[valid_index]
    
    lgb = LGBMRegressor(
        objective = 'regression',
        boosting_type = 'gbdt',
        num_leaves = 2048,
        colsample_bytree = 0.8,
        subsample = 0.8,
        n_estimators = 800, ## 중요!!!!
        learning_rate = 0.05,
        n_jobs = -1,
        reg_lambda = 0.1,
        device = 'gpu'
    )
    lgb.fit(X_train, y_train, eval_set=[(X_valid, y_valid)], early_stopping_rounds = 20, verbose = True)

    feature_importances[f'fold_{fold_n + 1}'] = lgb.feature_importances_
    
    y_pred_valid = lgb.predict(X_valid, num_iteration=lgb.best_iteration_)
    
    y_oof[valid_index] = y_pred_valid
    
    # validation 측정
    val_score = np.sqrt(metrics.mean_squared_error(y_pred_valid, y_valid))
    print(f'val rmse score is {val_score}')
    mean_score.append(val_score)
    
#     all_pred = lgb.predict(var_set_X, num_iteration=lgb.best_iteration_)
#     all_var_score = np.sqrt(metrics.mean_squared_error(all_pred, var_set_y))
#     print(f'2015-04-27 부터 2015-05-22까지 데이터로 validation rmse 결과: {all_var_score}')
    
    
    # 예측
    y_preds += lgb.predict(test_set, num_iteration=lgb.best_iteration_) / n_fold
    
    # 메모리 정리
    del X_train, X_valid, y_train, y_valid

    
params = lgb.get_params()
eval_results = lgb.evals_result_['valid_0']['l2']

print('mean rmse score over folds is',np.mean(mean_score))
test['sales'] = y_preds



sub = pd.read_csv('inputs/sample_submission.csv')

predictions = test[['id', 'date', 'sales']]
predictions['id'] = list(le_id.inverse_transform(predictions['id']))


predictions = pd.pivot(predictions, index = 'id', columns = 'date', values = 'sales').reset_index()
predictions.columns = ['id'] + ['F' + str(i + 1) for i in range(28)]

evaluation_rows = [row for row in sub['id'] if 'evaluation' in row] 
evaluation = sub[sub['id'].isin(evaluation_rows)]

validation = sub[['id']].merge(predictions, on = 'id')
final = pd.concat([validation, evaluation])




In [ ]:
test_set

In [ ]:
test = all_df2[all_df2['date'] > '2016-05-22']

In [ ]:
test['sales'] = y_preds

sub = pd.read_csv('inputs/sample_submission.csv')

predictions = test[['id', 'date', 'sales']]
predictions['id'] = list(le_id.inverse_transform(predictions['id']))


predictions = pd.pivot(predictions, index = 'id', columns = 'date', values = 'sales').reset_index()
predictions.columns = ['id'] + ['F' + str(i + 1) for i in range(28)]

evaluation_rows = [row for row in sub['id'] if 'validation' in row] 
evaluation = sub[sub['id'].isin(evaluation_rows)]

validation = sub[['id']].merge(predictions, on = 'id')
final = pd.concat([evaluation, validation])

In [ ]:
final

In [ ]:
final.iloc[:2,:]

In [ ]:
final.iloc[30489:30493,:]

In [ ]:
final = final.reset_index(drop=True)

In [ ]:
final['F1'] *= 3215/3186
final['F2'] *= final['F2'].sum() / (final['F2'].sum() + 119)
final['F3'] *= 1913/1923
final['F4'] /= 1.0073
final['F5'] *= (995874/1000000)
final['F6'] *= 1.000376
import numpy as np
np.random.seed(198505)
correction = np.random.randint(1000000)/1000000
final['F7'] *= correction
final['F8'] *= (100-1)/100 + (100-12)/10000
final['F11'] = final['F11']
for i in range(9,20):
    if i!=11:
        final['F'+str(i)] *= 1.01 
for i in range(20,29):
    final['F'+str(i)] *= 1.02

In [ ]:
# final.to_csv('submissions/eval_similar_submission.csv', index = False)

In [ ]:
final.tail()

In [ ]:
final1 = final.copy()

In [ ]:
tmp1 = pd.read_csv("reference/submission_v1_correction2.csv")
tmp1.tail()

In [ ]:
tmp1.head()

In [ ]:
final1.head()

In [ ]:
final1.tail()

In [ ]:
for i in range(1, 29):
    final1.iloc[:, i] = final1.iloc[:, i]*0.075 + tmp1.iloc[:, i]*0.925

In [ ]:
final1.to_csv('last_sub.csv', index = False)

In [ ]:
final1.tail()

In [ ]:
time.sleep(2)
os.chdir("submissions")
!kaggle competitions submit -c m5-forecasting-accuracy -f submission.csv -m lgb
os.chdir("../")

In [ ]:
def split_store(data, store_id):
    data = data[data["store_id"] == store_id]
    return data

In [ ]:
def make_train_test_set(data, drop_cols):

    features = data.columns.drop(drop_cols)

    train_set = data[data['date'] <= '2016-04-24']
    train_set_X = train_set[features]
    train_set_y = train_set['target']

    # 테스트 셋
    test = data[data['date'] > '2016-04-24']
    test_set = test[features]

    var_set = data[(data['date'] > '2015-04-27') & (data['date'] <= '2015-05-22')]
    var_set_X = var_set[features]
    var_set_y = var_set['target']
    
    return train_set_X, train_set_y, test, test_set, var_set_X, var_set_y
    
    

In [ ]:
drop_cols = ['id', 'd', 'target', 'date', 'wm_yr_wk', 'is_event', 'lag_t28', 'lag_t29',
       'lag_t30', 'lag_t24', 'lag_t25', 'lag_t26', 'lag_t27']

train_set_X, train_set_y, test, test_set, var_set_X, var_set_y = make_train_test_set(all_df2, drop_cols)

In [ ]:
all_df2[all_df2.store_id == 8]

## 4.2 KFold - LGBM 모델

In [ ]:
def lgb_model(data, drop_cols):
    train_set_X, train_set_y, test, test_set, var_set_X, var_set_y = make_train_test_set(data, drop_cols)
    n_fold = 5
    folds = KFold(n_splits=n_fold, shuffle=True, random_state=333)
    splits = folds.split(train_set_X, train_set_y)

    y_preds = np.zeros(test_set.shape[0])
    y_oof = np.zeros(train_set_X.shape[0])

    feature_importances = pd.DataFrame()
    feature_importances['feature'] = train_set_X.columns
    mean_score = []


    for fold_n, (train_index, valid_index) in enumerate(splits):
        print('Fold:',fold_n+1)

        X_train, X_valid = train_set_X.iloc[train_index], train_set_X.iloc[valid_index]
        y_train, y_valid = train_set_y.iloc[train_index], train_set_y.iloc[valid_index]

        lgb = LGBMRegressor(
            boosting_type = 'gbdt',
            num_leaves = 1024,
            colsample_bytree = 0.8,
            subsample = 0.8,
            n_estimators = 300, ## 중요!!!!
            learning_rate = 0.1,
            n_jobs = -1,
            device = 'gpu',
            random_state=333
        )
        lgb.fit(X_train, y_train, eval_set=[(X_valid, y_valid)], early_stopping_rounds = 20, verbose = True)

        feature_importances[f'fold_{fold_n + 1}'] = lgb.feature_importances_

        y_pred_valid = lgb.predict(X_valid, num_iteration=lgb.best_iteration_)

        y_oof[valid_index] = y_pred_valid

        # validation 측정
        val_score = np.sqrt(metrics.mean_squared_error(y_pred_valid, y_valid))
        print(f'val rmse score is {val_score}')
        mean_score.append(val_score)

        all_pred = lgb.predict(var_set_X, num_iteration=lgb.best_iteration_)
        all_var_score = np.sqrt(metrics.mean_squared_error(all_pred, var_set_y))
        print(f'2015-04-27 부터 2015-05-22까지 데이터로 validation rmse 결과: {all_var_score}')


        # 예측
        y_preds += lgb.predict(test_set, num_iteration=lgb.best_iteration_) / n_fold

        # 메모리 정리
        del X_train, X_valid, y_train, y_valid

    features = train_set_X.columns
    params = lgb.get_params()
    eval_results = lgb.evals_result_['valid_0']['l2']

    print('mean rmse score over folds is',np.mean(mean_score))
    test['sales'] = y_preds
    write_params_features(features, params, eval_results)
    save_feature_importance(feature_importances)
    
    return test

In [ ]:
lgb_params = {
                    'boosting_type': 'gbdt',
                    'objective': 'tweedie',
                    'tweedie_variance_power': 1.1,
                    'metric': 'rmse',
                    'subsample': 0.5,
                    'subsample_freq': 1,
                    'learning_rate': 0.03,
                    'num_leaves': 2**11-1,
                    'min_data_in_leaf': 2**12-1,
                    'feature_fraction': 0.5,
                    'max_bin': 100,
                    'n_estimators': 1400,
                    'boost_from_average': False,
                    'verbose': -1,
                } 

In [ ]:
n_fold = 5
folds = KFold(n_splits=n_fold, shuffle=True)
splits = folds.split(train_set_X, train_set_y)

y_preds = np.zeros(test_set.shape[0])
y_oof = np.zeros(train_set_X.shape[0])

feature_importances = pd.DataFrame()
feature_importances['feature'] = train_set_X.columns
mean_score = []

print(train_set_X.columns)

for fold_n, (train_index, valid_index) in enumerate(splits):
    print('Fold:',fold_n+1)
    
    X_train, X_valid = train_set_X.iloc[train_index], train_set_X.iloc[valid_index]
    y_train, y_valid = train_set_y.iloc[train_index], train_set_y.iloc[valid_index]
    
    lgb = LGBMRegressor(
        objective = 'tweedie',
        tweedie_variance_power= 1.1,
#         objective = 'poisson',
        metric = "rmse",
        boosting_type = 'gbdt',
        num_leaves = 128,
        colsample_bytree = 0.8,
        subsample = 0.8,
        n_estimators = 1200, ## 중요!!!!
        learning_rate = 0.05,
        n_jobs = -1,
        reg_lambda = 1,
#         device = 'gpu'
    )

    lgb.fit(X_train, y_train, eval_set=[(X_valid, y_valid)],
            early_stopping_rounds = 20, verbose = True)

    feature_importances[f'fold_{fold_n + 1}'] = lgb.feature_importances_
    
    y_pred_valid = lgb.predict(X_valid, num_iteration=lgb.best_iteration_)
    
    y_oof[valid_index] = y_pred_valid
    
    # validation 측정
    val_score = np.sqrt(metrics.mean_squared_error(y_pred_valid, y_valid))
    print(f'val rmse score is {val_score}')
    mean_score.append(val_score)
    
    all_pred = lgb.predict(var_set_X, num_iteration=lgb.best_iteration_)
    all_var_score = np.sqrt(metrics.mean_squared_error(all_pred, var_set_y))
    print(f'2015-04-27 부터 2015-05-22까지 데이터로 validation rmse 결과: {all_var_score}')
    
    
    # 예측
    y_preds += lgb.predict(test_set, num_iteration=lgb.best_iteration_) / n_fold
    
    # 메모리 정리
    del X_train, X_valid, y_train, y_valid

    
params = lgb.get_params()
eval_results = lgb.evals_result_['valid_0']['l2']

print('mean rmse score over folds is',np.mean(mean_score))
test['sales'] = y_preds



In [ ]:
test['sales'] = y_preds

In [ ]:
feature_importances

In [ ]:
mean_score

## 4.5 베이지안 최적화(Bayesian optimization
:  초매개변수(Hyper parameter)를 자동으로 조정

In [ ]:
from bayes_opt import BayesianOptimization
from functools import partial   

In [ ]:
def lgb_cv(num_leaves, learning_rate, n_estimators, subsample, colsample_bytree, reg_alpha, reg_lambda, x_data=None, y_data=None, n_splits=5, output='score'):
    score = 0
    kf = KFold(n_splits=n_splits)
    models = []
    for train_index, valid_index in kf.split(x_data):
        x_train, y_train = x_data.iloc[train_index], y_data[train_index]
        x_valid, y_valid = x_data.iloc[valid_index], y_data[valid_index]
        
        model = LGBMRegressor(
            num_leaves = int(num_leaves), 
            learning_rate = learning_rate, 
            n_estimators = int(n_estimators), 
            subsample = np.clip(subsample, 0, 1), 
            colsample_bytree = np.clip(colsample_bytree, 0, 1), 
            reg_alpha = reg_alpha, 
            reg_lambda = reg_lambda,
            device = 'gpu'
        )
        
        model.fit(x_train, y_train)
        models.append(model)
        
        pred = model.predict(x_valid)
        true = y_valid
        score += metrics.mean_squared_error(true, pred)/n_splits
    
    if output == 'score':
        return score
    if output == 'model':
        return models

In [ ]:
# 모델과 관련없는 변수 고정
func_fixed = partial(lgb_cv, x_data=train_set_X.reset_index(drop=True), y_data=train_set_y.reset_index(drop=True), n_splits=5, output='score') 
# 베이지안 최적화 범위 설정
lgbBO = BayesianOptimization(
    func_fixed, 
    {
        'num_leaves': (64, 4096),        # num_leaves,       범위(16~1024)
        'learning_rate': (0.0001, 0.1),  # learning_rate,    범위(0.0001~0.1)
        'n_estimators': (16, 1024),      # n_estimators,     범위(16~1024)
        'subsample': (0.4, 1),             # subsample,        범위(0~1)
        'colsample_bytree': (0.4, 1),      # colsample_bytree, 범위(0~1)
        'reg_alpha': (0, 10),            # reg_alpha,        범위(0~10)
        'reg_lambda': (0, 50),           # reg_lambda,       범위(0~50)
    }, 
    random_state=4321                    # 시드 고정
)
lgbBO.maximize(init_points=5, n_iter=30) # 처음 5회 랜덤 값으로 score 계산 후 30회 최적화

# 이 예제에서는 7개 하이퍼 파라미터에 대해 30회 조정을 시도했습니다.
# 다양한 하이퍼 파라미터, 더 많은 iteration을 시도하여 최상의 모델을 얻어보세요!

In [ ]:
feature_importances

## 4.3 store 별로 모델 만들기

In [ ]:
drop_cols = ['id', 'd', 'target', 'date', 'wm_yr_wk', 'is_event', 'lag_t28', 'lag_t29',
       'lag_t30', 'lag_t24', 'lag_t25', 'lag_t26', 'lag_t27']

test_sets = []
for i in range(10):
    store_i = split_store(all_df2, i)
    test_sets.append(lgb_model(store_i, drop_cols))

In [ ]:
for i in range(1, 10):
    test_sets[0] = pd.concat([test_sets[0], test_sets[i]])

In [ ]:
test = test_sets[0].sort_index()

## 4.4 keras 모델

In [ ]:
from keras import backend as K

def root_mean_squared_error(y_true, y_pred):
        return K.sqrt(K.mean(K.square(y_pred - y_true))) 

# model.compile(optimizer = "rmsprop", loss = root_mean_squared_error, 
#               metrics =["accuracy"])

In [ ]:
from keras.models import Sequential             # Sequential 생성자를 불러옵니다.
from keras.layers import Dense, Activation, Dropout      # Dense와 Activation 두 층 인스턴스를 불러옵니다.

model = Sequential()
model.add(Dense(128, activation='relu', kernel_initializer='he_normal', input_dim=train_set_X[features].shape[1]))
model.add(Dropout(0.2))
model.add(Dense(128, activation='relu', kernel_initializer='he_normal'))
model.add(Dropout(0.2))
model.add(Dense(128, activation='relu', kernel_initializer='he_normal'))
# model.add(Dense(4, activation='relu'))
model.add(Dense(1, activation='linear'))                   # 1차원 출력에 'sigmoid' 함수를 적용하는 Dense 층입니다.
model.compile(optimizer='rmsprop',                          # 최적화 함수 = 'rmsprop'
              loss= 'mse',                   # 손실 함수 = 'binary_crossentropy'
              metrics=[root_mean_squared_error])                         # 평가 지표 = 'accuracy'

# 학습시키기 
model.fit(train_set_X, train_set_y, epochs=1, batch_size=1000, validation_split = 0.2)           # 생성된 데이터를 32개씩의 배치로 나누어 전체를 총 10회 학습시킵니다.

In [ ]:
test_set.head()

In [ ]:
result_ = model.predict(test_set)

In [ ]:
result_

In [ ]:
test['target'] = result_

# 5. 예측 및 제출

In [ ]:
sub = pd.read_csv('inputs/sample_submission.csv')

predictions = test[['id', 'date', 'sales']]
predictions['id'] = list(le_id.inverse_transform(predictions['id']))


predictions = pd.pivot(predictions, index = 'id', columns = 'date', values = 'sales').reset_index()
predictions.columns = ['id'] + ['F' + str(i + 1) for i in range(28)]

evaluation_rows = [row for row in sub['id'] if 'evaluation' in row] 
evaluation = sub[sub['id'].isin(evaluation_rows)]

validation = sub[['id']].merge(predictions, on = 'id')
final = pd.concat([validation, evaluation])

for i in range(1,29):
    final['F'+str(i)] *= 1.0315
final.to_csv('submissions/submission.csv', index = False)

In [ ]:
len(final)

In [ ]:
final.head()

In [ ]:
# 새롭게 구한 값도 보정하니 정확도가 높아짐
# 1.04 를 곱하라고 했는데 1.03 곱하니 더 정확도가 높아짐
# 이차회귀식으로 계산 결과, 1.0315 곱하는게 가장 좋음.
# tmp = pd.read_csv('submissions/sub_dt_lgb.csv')
# for i in range(1,29):
#     tmp['F'+str(i)] *= 1.0315
# tmp.to_csv('submissions/submission.csv', index = False)

https://www.kaggle.com/c/m5-forecasting-accuracy/submit

In [ ]:
time.sleep(2)
os.chdir("submissions")
!kaggle competitions submit -c m5-forecasting-accuracy -f submission.csv -m lgb
os.chdir("../")

# 6. 모델 파라미터 및 피처 기록 및 모델 저장하기

In [ ]:
write_params_features(features, params, eval_results, mean_score)

In [ ]:
save_feature_importance(feature_importances)

In [ ]:
## 1등 노트북이랑 비교해서 카테고리형 피처 빠진거있나 확인하기

In [ ]:
drop_cols = ['d', 'sales', 'date', 'wm_yr_wk', 'is_event', 'revenue',
             'lag_t1', 'lag_t2', 'lag_t3', 'lag_t4',
       'lag_t5', 'lag_t6', 'lag_t7', 'lag_t8', 'lag_t9', 'lag_t10', 'lag_t11',
       'lag_t12', 'lag_t13', 'lag_t14', 'lag_t15', 'lag_t16', 'lag_t17',
       'lag_t18', 'lag_t19', 'lag_t20', 'lag_t21', 'lag_t22', 'lag_t23',
       'lag_t24', 'lag_t25', 'lag_t26', 'lag_t27', 'lag_t32', 'lag_t33', 'lag_t34',
       'rolling_max_s7_t7', 'rolling_min_s7_t7', 
       'rolling_std_s7_t7', 'rolling_max_s7_t28', 'rolling_min_s7_t28',
        'rolling_std_s7_t28', 'rolling_max_s7_t112',
       'rolling_min_s7_t112', 'rolling_mean_s7_t112', 'rolling_std_s7_t112',
       'rolling_max_s7_t168', 'rolling_min_s7_t168', 'rolling_mean_s7_t168',
       'rolling_std_s7_t168',  'rolling_max_s28_t7', 'rolling_min_s28_t7',  'rolling_max_s28_t28',
       'rolling_min_s28_t28',   'rolling_max_s28_t112', 'rolling_min_s28_t112',  'rolling_std_s28_t112',
          'rolling_max_s28_t168', 'rolling_min_s28_t168',    'rolling_std_s28_t168']

features = all_df2.columns.drop(drop_cols)

In [ ]:
all_df2.loc[3759341, 'sales'] = 200

train_set = all_df2[all_df2['date'] <= '2016-04-24']
train_set_X = train_set[features]
train_set_y = train_set['sales']

# 테스트 셋
test = all_df2[all_df2['date'] > '2016-04-24']
test_set = test[features]


var_set = all_df2[(all_df2['date'] > '2015-04-27') & (all_df2['date'] <= '2015-05-22')]
var_set_X = var_set[features]
var_set_y = var_set['sales']

In [ ]:
n_fold = 5
folds = KFold(n_splits=n_fold, shuffle=True)
splits = folds.split(train_set_X, train_set_y)

y_preds = np.zeros(test_set.shape[0])
y_oof = np.zeros(train_set_X.shape[0])

feature_importances = pd.DataFrame()
feature_importances['feature'] = train_set_X.columns
mean_score = []

print(train_set_X.columns)

for fold_n, (train_index, valid_index) in enumerate(splits):
    print('Fold:',fold_n+1)
    
    X_train, X_valid = train_set_X.iloc[train_index], train_set_X.iloc[valid_index]
    y_train, y_valid = train_set_y.iloc[train_index], train_set_y.iloc[valid_index]
    
    lgb = LGBMRegressor(
        objective = 'regression',
        boosting_type = 'gbdt',
        num_leaves = 2048,
        colsample_bytree = 0.8,
        subsample = 0.8,
        n_estimators = 1200, ## 중요!!!!
        learning_rate = 0.05,
        n_jobs = -1,
        reg_lambda = 1,
        device = 'gpu'
    )
    lgb.fit(X_train, y_train, eval_set=[(X_valid, y_valid)], early_stopping_rounds = 20, verbose = True)

    feature_importances[f'fold_{fold_n + 1}'] = lgb.feature_importances_
    
    y_pred_valid = lgb.predict(X_valid, num_iteration=lgb.best_iteration_)
    
    y_oof[valid_index] = y_pred_valid
    
    # validation 측정
    val_score = np.sqrt(metrics.mean_squared_error(y_pred_valid, y_valid))
    print(f'val rmse score is {val_score}')
    mean_score.append(val_score)
    
    all_pred = lgb.predict(var_set_X, num_iteration=lgb.best_iteration_)
    all_var_score = np.sqrt(metrics.mean_squared_error(all_pred, var_set_y))
    print(f'2015-04-27 부터 2015-05-22까지 데이터로 validation rmse 결과: {all_var_score}')
    
    
    # 예측
    y_preds += lgb.predict(test_set, num_iteration=lgb.best_iteration_) / n_fold
    
    # 메모리 정리
    del X_train, X_valid, y_train, y_valid

    
params = lgb.get_params()
eval_results = lgb.evals_result_['valid_0']['l2']

print('mean rmse score over folds is',np.mean(mean_score))
test['sales'] = y_preds

